<a href="https://colab.research.google.com/github/aarijmh/chair_mapper/blob/main/NewTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Install FiftyOne and Ultralytics
!pip -q install fiftyone ultralytics

from google.colab import drive
drive.mount('/content/drive')

import random, shutil
from pathlib import Path
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone.utils import openimages as fouo


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:

/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'


In [2]:

# Paths
DRIVE = Path("/content/drive/MyDrive")
NEG = DRIVE / "yolo11_monitor_negatives"     # your existing negatives
POS_DESKTOP_OUT = DRIVE / "yolo11_monitor_positives_desktop"  # refined positives (desktop-only)
FULL_OUT = DRIVE / "yolo11_monitor_full"

# Open Images classes
MONITOR = "Computer monitor"
LAPTOP  = "Laptop"

# Target size and splits
TOTAL_SAMPLES = 800              # adjust as needed
SPLITS_ORDER  = ["validation", "test", "train"]  # will fallback to train if needed
TRAIN_RATIO   = 0.8
VAL_RATIO     = 0.2
TEST_RATIO    = 0.0

# Overlap threshold to consider a monitor box "too close" to a laptop box (drop sample)
LAPTOP_IOU_THRESHOLD = 0.30


In [3]:

# Validate class names
oi_classes = set(fouo.get_classes(version="v7"))
assert MONITOR in oi_classes and LAPTOP in oi_classes, "Class names not found in Open Images V7"

def load_subset(max_samples, splits=SPLITS_ORDER, seed=42):
    """
    Load Open Images V7 images that contain either Computer monitor or Laptop.
    We'll filter them after load.
    """
    remaining = max_samples
    datasets = []

    for split in splits:
        if remaining <= 0:
            break
        print(f"Loading split='{split}' with classes [{MONITOR}, {LAPTOP}]")
        try:
            ds = foz.load_zoo_dataset(
                "open-images-v7",
                split=split,
                label_types=["detections"],
                classes=[MONITOR, LAPTOP],  # load both to allow laptop filtering
                only_matching=True,
                max_samples=remaining,
                shuffle=True,
                seed=seed,
            )
            if len(ds) > 0:
                print("  ->", len(ds), "samples")
                datasets.append(ds)
                remaining -= len(ds)
            else:
                print("  -> zero samples")
        except Exception as e:
            print("  !! error:", e)

    if not datasets:
        return None

    merged = datasets[0]
    for d in datasets[1:]:
        merged.merge_samples(d)
        d.delete()
    return merged

ds = load_subset(TOTAL_SAMPLES)
assert ds is not None and len(ds) > 0, "No samples loaded; try increasing TOTAL_SAMPLES or include 'train' split"

print("Loaded:", len(ds))


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/tmp/tmp8j_lxsa2/metadata/classes.csv'


Loading split='validation' with classes [Computer monitor, Laptop]


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/open-images-v7/validation' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/validation/validation-images-with-rotation.csv' to '/root/fiftyone/open-images-v7/validation/metadata/image_ids.csv'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/validation/metadata/classes.csv'


...

Converting existing index 'filepath' to unique on dataset 'open-images-v7-validation-800'


INFO:fiftyone.core.collections:Converting existing index 'filepath' to unique on dataset 'open-images-v7-validation-800'


Converting existing index 'filepath' to unique on dataset 'open-images-v7-train-323'


INFO:fiftyone.core.collections:Converting existing index 'filepath' to unique on dataset 'open-images-v7-train-323'


Loaded: 800


In [4]:

def has_label(sample, label_name):
    dets = sample.get_field("ground_truth")
    if dets is None:
        return False
    return any(d.label == label_name for d in dets.detections)

def iou_xywh(b1, b2):
    """
    IoU for relative [x, y, w, h] in image coordinates (normalized).
    """
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2

    # Convert to [x1,y1,x2,y2]
    ax1, ay1, ax2, ay2 = x1, y1, x1 + w1, y1 + h1
    bx1, by1, bx2, by2 = x2, y2, x2 + w2, y2 + h2

    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)

    iw, ih = max(0.0, ix2 - ix1), max(0.0, iy2 - iy1)
    inter = iw * ih
    area1 = w1 * h1
    area2 = w2 * h2
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0

# Keep only samples that contain MONITOR and do NOT contain LAPTOP
keep_ids = []
for s in ds:
    if has_label(s, MONITOR) and not has_label(s, LAPTOP):
        keep_ids.append(s.id)

filtered = ds.select(keep_ids)
print(f"After 'no laptop' filter: {len(filtered)} samples")

# IoU veto: drop samples where all monitor boxes strongly overlap any laptop box (defensive)
veto_ids = []
for s in filtered:
    dets = s.get_field("ground_truth")
    monitors = [d for d in dets.detections if d.label == MONITOR]
    laptops  = [d for d in dets.detections if d.label == LAPTOP]
    if not monitors:
        continue
    if laptops:
        # If every monitor overlaps some laptop ≥ threshold, veto sample
        all_overlap = True
        for m in monitors:
            if not any(iou_xywh(m.bounding_box, l.bounding_box) >= LAPTOP_IOU_THRESHOLD for l in laptops):
                all_overlap = False
                break
        if all_overlap:
            veto_ids.append(s.id)

if veto_ids:
    filtered = filtered.exclude(veto_ids)
    print(f"After IoU veto ({LAPTOP_IOU_THRESHOLD}): removed {len(veto_ids)}, kept {len(filtered)}")
else:
    print("No IoU veto applied (no overlapping laptop boxes).")


After 'no laptop' filter: 228 samples
No IoU veto applied (no overlapping laptop boxes).


In [6]:
random.seed(42)
ids = [s.id for s in filtered]
random.shuffle(ids)

n = len(ids)
n_train = int(n * TRAIN_RATIO)
n_val   = int(n * VAL_RATIO)
n_test  = max(0, n - n_train - n_val) if TEST_RATIO == 0 else int(n * TEST_RATIO)

train_view = filtered.select(ids[:n_train])
val_view   = filtered.select(ids[n_train:n_train+n_val])
test_view  = filtered.select(ids[n_train+n_val:n_train+n_val+n_test]) if n_test > 0 else None

# Export YOLO (class index 0 -> Computer monitor)
classes = [MONITOR]
POS_DESKTOP_OUT.mkdir(parents=True, exist_ok=True)

train_view.export(
    export_dir=str(POS_DESKTOP_OUT),
    dataset_type=fo.types.YOLOv5Dataset,   # YOLO format compatible with YOLOv11
    label_field="ground_truth",
    split="train",
    classes=classes,
    export_media=True,
)
val_view.export(
    export_dir=str(POS_DESKTOP_OUT),
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    split="val",
    classes=classes,
    export_media=True,
)
if test_view:
    test_view.export(
        export_dir=str(POS_DESKTOP_OUT),
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
        split="test",
        classes=classes,
        export_media=True,
)

print("Desktop-only positives exported to:", POS_DESKTOP_OUT)

Directory '/content/drive/MyDrive/yolo11_monitor_positives_desktop' already exists; export will be merged with existing files


 100% |█████████████████| 182/182 [5.8s elapsed, 0s remaining, 38.0 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 182/182 [5.8s elapsed, 0s remaining, 38.0 samples/s]      


Directory '/content/drive/MyDrive/yolo11_monitor_positives_desktop' already exists; export will be merged with existing files


 100% |███████████████████| 45/45 [1.4s elapsed, 0s remaining, 31.5 samples/s]         


INFO:eta.core.utils: 100% |███████████████████| 45/45 [1.4s elapsed, 0s remaining, 31.5 samples/s]         


Directory '/content/drive/MyDrive/yolo11_monitor_positives_desktop' already exists; export will be merged with existing files


 100% |█████████████████████| 1/1 [33.7ms elapsed, 0s remaining, 29.7 samples/s] 


INFO:eta.core.utils: 100% |█████████████████████| 1/1 [33.7ms elapsed, 0s remaining, 29.7 samples/s] 


Desktop-only positives exported to: /content/drive/MyDrive/yolo11_monitor_positives_desktop


In [7]:

def copy_pair(img_src, lbl_src, img_dst_dir, lbl_dst_dir, prefix=None):
    stem = img_src.stem if prefix is None else f"{prefix}_{img_src.stem}"
    dst_img = Path(img_dst_dir) / (stem + img_src.suffix.lower())
    dst_lbl = Path(lbl_dst_dir) / (stem + ".txt")
    if not dst_img.exists():
        shutil.copy2(img_src, dst_img)
    if lbl_src and lbl_src.exists():
        if not dst_lbl.exists():
            shutil.copy2(lbl_src, dst_lbl)
    else:
        dst_lbl.touch()

def copy_split(src_root, split, dst_root, prefix):
    img_dir = src_root / f"images/{split}"
    lbl_dir = src_root / f"labels/{split}"
    if not img_dir.exists():
        return 0
    files = [p for p in img_dir.glob("*") if p.suffix.lower() in {".jpg",".jpeg",".png"}]
    for img in files:
        lbl = lbl_dir / (img.stem + ".txt")
        copy_pair(img, lbl if lbl_dir.exists() else None,
                  dst_root / f"images/{split}",
                  dst_root / f"labels/{split}",
                  prefix=prefix)
    return len(files)

# Create output tree
for sub in ["images/train","images/val","images/test","labels/train","labels/val","labels/test"]:
    (FULL_OUT / sub).mkdir(parents=True, exist_ok=True)

merged_counts = {}
for split in ["train","val","test"]:
    pos_n = copy_split(POS_DESKTOP_OUT, split, FULL_OUT, prefix="pos")
    neg_n = copy_split(NEG, split, FULL_OUT, prefix="neg")
    merged_counts[split] = {"pos": pos_n, "neg": neg_n}

print("Merged:", merged_counts)

# Write dataset.yaml
import yaml
dataset_yaml = {
    "path": str(FULL_OUT),
    "train": "images/train",
    "val": "images/val",
    "test": "images/test" if any((FULL_OUT / "images/test").glob("*")) else None,
    "names": [MONITOR]  # single class
}
dataset_yaml = {k: v for k, v in dataset_yaml.items() if v is not None}
with open(FULL_OUT / "dataset.yaml", "w") as f:
    yaml.safe_dump(dataset_yaml, f, sort_keys=False)

print("Wrote:", FULL_OUT / "dataset.yaml")


Merged: {'train': {'pos': 182, 'neg': 160}, 'val': {'pos': 45, 'neg': 40}, 'test': {'pos': 1, 'neg': 0}}
Wrote: /content/drive/MyDrive/yolo11_monitor_full/dataset.yaml
